In [213]:
import json
import pprint
import collections
import re

split_words = lambda s: s.split() 
strip_space = lambda s: re.sub("\s+", " ", re.sub("\s+\?", "?", re.sub("\s+\.", ".", re.sub("\s+,", ".", s)))).lstrip().rstrip().replace("..", ".")

def load_examples(file_name: str):
    ans = []
    err = collections.defaultdict(int)
    with open(file_name, 'r') as handler:
        for line in handler:
            example = json.loads(line)
            question = example['question']
            example = json.loads(line)
            question = example['question']
            
            #if example['id'] != 'QuaRel_V1_B4_0325':
            #    continue
            
            if "(A)" not in question or "(B)" not in question:
                err['A_B'] += 1
                continue 
                
            idx = question.index("(A)")
            Q, second = question[0:idx], question[idx:]
                
            ans_idx = second.index("(B)")
            ans_first, ans_second = second[0:ans_idx].replace("(A)", ""), second[ans_idx:].replace("(B)", "")
            ans_first = re.sub("or\s*$", "", ans_first)
            ans_second = re.sub("or\s*$", "", ans_second)
            
            A = ans_first if example.get("answer_index", 0) == 0 else ans_second
            
            if "_____" in Q:
                Q = Q.replace("_____", A)
            else:
                Q = "{0} {1}.".format(Q, A)
            
            # split it into two sentence
            Q = re.sub(" +", " ", Q)
            
            Q_items = re.split("(\.|\?            print(Q_2))", Q)
            Q_items_process = []
            for i in range(len(Q_items)):
                if Q_items[i] == '.' or Q_items[i] == '?':
                    Q_items_process[-1] = "{0}{1}".format(Q_items_process[-1], Q_items[i])
                else:
                    if Q_items[i]:
                        Q_items_process.append(Q_items[i])
    
            Q_items = [i for i in Q_items_process if i]
 
            Q_1, Q_2 = [], []
            threshold, tmp_w = 5, 0
            for idx, s in enumerate(Q_items[::-1]):
                if tmp_w <= threshold:
                    Q_2.append(s)
                else:
                    Q_1.append(s)
                tmp_w += len(split_words(s))
                            
            Q_1 = " ".join(Q_1[::-1]).rstrip()
            Q_2 = " ".join(Q_2[::-1]).rstrip()
            if not Q_1:
                Qs = re.split("(it\s+\w+|due to|so|because|and\s+\w+|it became|and it|this means|since|\?\s+|\.\s+|,\s+|but|it appeared)", Q_2)
                Q_1, Q_2 = " ".join(Qs[0:-2]), " ".join(Qs[-2:])
                Q_2 = re.sub(",\s+|\.\s+|\?\s+", "", Q_2)
            
            if not Q_2:
                err["Q_2"] += 1
            
            if not Q_1:
                err["Q_1"] += 1
            
            item = {
                "Q": question,
                "id": example['id'],
                "Q_1": strip_space(Q_1),
                "Q_2": strip_space(Q_2),
            }
            
            ans.append(item)
    print(err)
    return ans

ans = load_examples(join(BASE_PATH, "quarel-data", "quarel-v1-train.json"))

w = open("corpus.txt", "w")
for a in ans:
    if not a["Q_1"]: 
        continue
        
    #w.write("{0}\n".format(a['id']))
    #w.write("{0}\n".format(a['Q']))
    w.write("{0}\n".format(a['Q_1']))
    w.write("{0}\n\n".format(a['Q_2']))
    #w.write("========\n\n")
w.close()

# pprint.pprint(ans[10:20])

defaultdict(<class 'int'>, {'Q_1': 14})


In [96]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import json
import numpy as np
from os.path import abspath, dirname, join

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# BASE_PATH = abspath(dirname(abspath(__file__)))
BASE_PATH = "/home/exx/Documents/Weihai/tutorial/bert-proj/"

# Load pre-trained model tokenizer (vocabulary)c
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/exx/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [45]:
# Load pre-trained model (weights)
# model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/exx/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/exx/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpe2lof4f7
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_pretrained_bert.modeling:Weights from pretrained mo

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [56]:
# Try the mask model
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim [MASK] was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

text = "[CLS] Mike was snowboarding on the snow and hit a piece of ice. He went much faster on the ice [SEP] because [MASK] is smoother. [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
# masked_index = 8
# tokenized_text[masked_index] = '[MASK]'
# print(tokenized_text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

['[CLS]', 'mike', 'was', 'snow', '##boarding', 'on', 'the', 'snow', 'and', 'hit', 'a', 'piece', 'of', 'ice', '.', 'he', 'went', 'much', 'faster', 'on', 'the', 'ice', '[SEP]', 'because', '[MASK]', 'is', 'smooth', '##er', '.', '[SEP]']
and


In [214]:
# try the predict next sentence model
# https://github.com/huggingface/pytorch-pretrained-BERT/issues/48
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM,BertForNextSentencePrediction,BertForSequenceClassification
import torch.nn.functional as F

model_name = 'bert-base-uncased'
model_name = './pytorch-bert/finetuned_lm/'
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
text = "A car gets very hot as it drives up a muddy hill, but stays cool as it drives up a grass hill. The car warms on on the muddy hill because the muddy hill has ', 'R_Q': '[CLS] A car gets very hot as it drives up a muddy hill, but stays cool as it drives up a grass hill. The car warms on on the muddy hill [SEP] because the muddy hill has less friction [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertForNextSentencePrediction.from_pretrained(model_name)
model.eval()

print("START")
# Predict is Next Sentence ?
predictions = model(tokens_tensor, segments_tensors )

print(predictions)
print(F.softmax(predictions))
# https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/pytorch_pretrained_bert/modeling.py
print("END")

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file ./pytorch-bert/finetuned_lm/vocab.txt
INFO:pytorch_pretrained_bert.modeling:loading archive file ./pytorch-bert/finetuned_lm/
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



['a', 'car', 'gets', 'very', 'hot', 'as', 'it', 'drives', 'up', 'a', 'muddy', 'hill', ',', 'but', 'stays', 'cool', 'as', 'it', 'drives', 'up', 'a', 'grass', 'hill', '.', 'the', 'car', 'warm', '##s', 'on', 'on', 'the', 'muddy', 'hill', 'because', 'the', 'muddy', 'hill', 'has', "'", ',', "'", 'r', '_', 'q', "'", ':', "'", '[', 'cl', '##s', ']', 'a', 'car', 'gets', 'very', 'hot', 'as', 'it', 'drives', 'up', 'a', 'muddy', 'hill', ',', 'but', 'stays', 'cool', 'as', 'it', 'drives', 'up', 'a', 'grass', 'hill', '.', 'the', 'car', 'warm', '##s', 'on', 'on', 'the', 'muddy', 'hill', '[SEP]', 'because', 'the', 'muddy', 'hill', 'has', 'less', 'friction', '[SEP]']


INFO:pytorch_pretrained_bert.modeling:Weights from pretrained model not used in BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']


START
tensor([[ 7.1011, -6.1035]], grad_fn=<AddmmBackward>)
tensor([[1.0000e+00, 1.8422e-06]], grad_fn=<SoftmaxBackward>)
END


/home/exx/anaconda3/envs/python_37/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
